In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.externals import joblib
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog


%matplotlib inline

In [2]:
def bin_spatial(img):    
    features = cv2.resize(img, (32,32)).ravel()
    return features

In [ ]:
from sklearn.cross_validation import train_test_split
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        if(image.shape[2] == 4):
            image = cv2.cvtColor(image,cv2.COLOR_RGBA2RGB)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        spatial = bin_spatial(image)

        # Append the new feature vector to the features list
        features.append(np.hstack((hog_features,spatial)))
    # Return list of feature vectors
    return features

colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

t=time.time()
cars = []
notCars = []

# add images of vehicles twice
for _ in range(2):
    cars1 = glob.glob('training_data/vehicles/part1/*.png')
    cars = cars + cars1
    cars1 = glob.glob('training_data/vehicles/part2/*.png')
    cars = cars + cars1
    cars1 = glob.glob('training_data/vehicles/part3/*.png')
    cars = cars + cars1
    cars1 = glob.glob('training_data/vehicles/part4/*.png')
    cars = cars + cars1
    cars1 = glob.glob('training_data/vehicles/part5/*.png')
    cars = cars + cars1

notCars1 = glob.glob('training_data/non-vehicles/part1/*.png')
notCars = notCars + notCars1
notCars1 = glob.glob('training_data/non-vehicles/part2/*.png')
notCars = notCars + notCars1
notCars1 = glob.glob('training_data/non-vehicles/part3/*.png')
notCars = notCars + notCars1
notCars1 = glob.glob('training_data/non-vehicles/part4/*.png')
notCars = notCars + notCars1

# to handle certain false positives, add certain non vehicle images 3 times
for _ in range(3):
    notCars1 = glob.glob('training_data/non-vehicles/part5/*.png')
    notCars = notCars + notCars1

car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notCars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
print("cf", np.array(car_features).shape)
print("ncf", np.array(notcar_features).shape)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)
print(X.shape)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
print("Saving model")

joblib.dump(svc, 'vehicle_detection.pkl')
joblib.dump(X_scaler, 'vehicle_detection_x_scaler.pkl')


In [3]:
# use the saved model
svc = joblib.load('vehicle_detection.pkl') 
X_scaler = joblib.load('vehicle_detection_x_scaler.pkl')

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(labels):
    boxesArr = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        boxesArr.append(bbox)
    return boxesArr
   
heatmap_prev = None
def find_car(img):
    ystart = 400
    ystop = 670
    xstart = 680
    xend = 1280
    scale = 1.2
    pix_per_cell = 8
    cell_per_block = 2
    orient = 9
    
    box_list = []
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255

    img_tosearch = img[ystart:ystop,xstart:xend,:]

    ctrans_tosearch = img_tosearch
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2

    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1

    hog1 = hog(ch1, orientations=orient, 
                   pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block), 
                   transform_sqrt=False, 
                   visualise=False, feature_vector=False)
    hog2 = hog(ch2, orientations=orient, 
                   pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block), 
                   transform_sqrt=False, 
                   visualise=False, feature_vector=False)
    hog3 = hog(ch3, orientations=orient, 
                   pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block), 
                   transform_sqrt=False, 
                   visualise=False, feature_vector=False) 

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step

            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1,hog_feat2,hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

            spatial = bin_spatial(subimg)
            test_features = X_scaler.transform(np.hstack((hog_features, spatial)).reshape(1, -1))

            confLevel = svc.decision_function(test_features)
            # filter out images whose distance is less then 0.3 from decision line
            if confLevel >= 0.3:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                box_list.append(((xbox_left+xstart, ytop_draw+ystart),(xbox_left+xstart+win_draw,ytop_draw+win_draw+ystart)))
  
    heat = np.zeros_like(draw_img[:,:,0]).astype(np.float)

    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,3)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # average the heatmap
    global heatmap_prev
    if heatmap_prev is not None:
        heatmap = (heatmap+heatmap_prev)//2.0
        
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    boxes = draw_labeled_bboxes(labels)
    
    for box in boxes:
        x1 = box[0][0]
        y1 = box[0][1]
        x2 = box[1][0]
        y2 = box[1][1]
        
        if(x1==x2):
            continue
        if(y1==y2):
            continue

        # Draw bounded box only if the confidence in the
        # bounded box is greater then 6.
        # This filters out the boxes that has low confidence.
        maxval = np.max(heatmap[y1:y2, x1:x2])
        if(maxval > 6):
            cv2.rectangle(draw_img, box[0], box[1], (0,0,255), 6)

        # All bounded boxes confidence increases by 2.
        # This helps filter out bounded boxes that only exists in 1 frame.
        heatmap[y1:y2, x1:x2] += 2
    
    # Filter out all the low confidence.
    # Anything lower than 5 is zeroed out.
    heatmap[heatmap <= 5] = 0
    heatmap_prev = heatmap
    
    return draw_img

In [4]:
import os
files = os.listdir("test_images/")
for file in files:
    if(file.startswith('.')):
        continue
    heatmap_prev = None
    img = mpimg.imread('test_images/' + file)
    result = find_car(img)
    mpimg.imsave('output_images/' + file, result)

/Users/eric/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [5]:
def process_image(img):
    finalImg = find_car(img)
    return finalImg

In [6]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'videoOutput/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video videoOutput/project_video.mp4
[MoviePy] Writing video videoOutput/project_video.mp4


100%|█████████▉| 1260/1261 [04:10<00:00,  4.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videoOutput/project_video.mp4 

CPU times: user 3min 59s, sys: 14.2 s, total: 4min 13s
Wall time: 4min 11s
